Ngram Model

In [19]:
# code courtesy of https://nlpforhackers.io/language-models/
from nltk.corpus import reuters
# from nltk import bigrams, trigrams
from nltk import ngrams
from collections import Counter, defaultdict
import itertools 

In [20]:
def getAlarmsSeqs(fname):
    lines = None
    with open(fname) as f:
        lines = f.readlines()

    temp_alarms_seqs = []
    for i,line in enumerate(lines):
        seq = line.split()
        temp_alarms_seqs.append(seq)
    
    temp_alarms_seqs =  list(itertools.chain(*temp_alarms_seqs)) 


    return temp_alarms_seqs
      

In [21]:
dir = "/Users/hamzagorgulu/Desktop/thesis/Waris Final/tupras/data/"
fname = dir + "train_filtered.tokens"
train_alarms_seqs = getAlarmsSeqs(fname)

# train_alarms_seqs[0:50]

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))
n = 12
for tup  in ngrams(train_alarms_seqs, n=n, pad_right=True, pad_left=True):
    # print(type(tup[0:n-1]),len(tup[0:n-1]))
    # print(a1,a2,a3,a4,a5)
    model[tup[0:n-1]][tup[-1]] += 1
 
# Let's transform the counts to probabilities
for key in model:
    # print(key)
    total_count = float(sum(model[key].values()))
    for w3 in model[key]:
        model[key][w3] /= total_count

In [22]:
# d = dict(model[ '47TI2086',
#  '47PDI003',
#  '47TI3418',
#  '47LI003',
#  '47LI003'])

# print(list(d.keys())[0],list(d.values())[0]) 

# print(d)

In [23]:
# calculate the accuracy
fname = dir + "val_filtered.tokens"
val_alarms_seqs = getAlarmsSeqs(fname)
alarm2acc = {}

for a in train_alarms_seqs:
    alarm2acc[a] = {"true":0, "false":0, "acc":0}

for a in val_alarms_seqs:
    alarm2acc[a] = {"true":0, "false":0, "acc":0}
 


for i in range(len(val_alarms_seqs)-n):
    seq = train_alarms_seqs[i:i+n]
    # print("seq =>",seq)
    
    in_seq = seq[0:n-1]
    y_true = seq[-1]
    # print(in_seq,y_true)
    # print(seq_in,y_true)
    
    d = dict(model[tuple(in_seq)])
    y_pred = list(d.keys())[0] 
    if y_pred == y_true:
       alarm2acc[y_true]['true'] += 1
    else:
       alarm2acc[y_true]['false'] += 1  
        # print(list(d.keys())[0],list(d.values())[0]) 


for a in alarm2acc.keys():
    if alarm2acc[a]["true"]+alarm2acc[a]["false"]>0: 
        alarm2acc[a]["acc"] =alarm2acc[a]["true"] /(alarm2acc[a]["true"]+alarm2acc[a]["false"]) 

more_than_50_percent_acc = [(a,alarm2acc[a]["acc"]) for a in alarm2acc.keys() if alarm2acc[a]["acc"]>=0.5]



print(f"Total number of alarms= {len(alarm2acc.keys())}")
print(f"Number of Alarm Sources with more than 50% Accuracy= {len(more_than_50_percent_acc)}")
# alarm2acc

Total number of alarms= 218
Number of Alarm Sources with more than 50% Accuracy= 209


In [24]:

for x in [0.5,0.6,0.7,0.8,0.9]:
    more_than_x_percent_acc = [(a,alarm2acc[a]["acc"]) for a in alarm2acc.keys() if alarm2acc[a]["acc"]>=x]

    print(f"Number of Alarm Sources with more than {x}% Accuracy = {len(more_than_x_percent_acc)} (%{round(100*(len(more_than_x_percent_acc))/(len(alarm2acc.keys())),2)})")

Number of Alarm Sources with more than 0.5% Accuracy = 209 (%95.87)
Number of Alarm Sources with more than 0.6% Accuracy = 208 (%95.41)
Number of Alarm Sources with more than 0.7% Accuracy = 208 (%95.41)
Number of Alarm Sources with more than 0.8% Accuracy = 201 (%92.2)
Number of Alarm Sources with more than 0.9% Accuracy = 142 (%65.14)
